In [2]:
import pandas as pd

# Data SUS

Um breve texto explicando o que é o DataSUS e o que é esperado dos dados.

## Extração de dados

In [3]:
df = pd.read_csv('resultado.csv', sep=';', encoding='latin1')

## Entendimento dos Dados

In [4]:
df

,Código Município,Município,Ações coletivas/individuais em saúde,Coleta de material,Diagnóstico em laboratório clínico,Diagnóstico por anatomia patológica e citopatologia,Diagnóstico por radiologia,Diagnóstico por ultrasonografia,Diagnóstico por tomografia,Diagnóstico por ressonância magnética,...,"Transplante de orgãos, tecidos e células",Acompanhamento e intercorrências no pré e pós-transplante,Medicamentos de âmbito hospitalar e urgência,"Órteses, próteses e materiais especiais relacionados ao ato cirúrgico",Ações relacionadas ao estabelecimento,Ações relacionadas ao atendimento,Total,Mes,Ano,Conteudo
0,110001,ALTA FLORESTA D'OESTE,0,0,580,0,26,1,0,0,...,0,0,0,0,0,173,1444,Dez,2023,Quantidade aprovada
1,110002,ARIQUEMES,0,51,11,0,143,9,54,0,...,0,0,6,20,0,1689,11710,Dez,2023,Quantidade aprovada
2,110003,CABIXI,0,0,48,0,1,0,0,0,...,0,0,0,0,0,0,62,Dez,2023,Quantidade aprovada
3,110004,CACOAL,0,3,9523,0,415,290,118,11,...,0,0,6,90,111,3714,44571,Dez,2023,Quantidade aprovada
4,110005,CEREJEIRAS,0,0,0,0,80,2,0,0,...,0,0,0,0,9,287,1268,Dez,2023,Quantidade aprovada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2842,522170,URUANA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,32,Dez,2023,Quantidade aprovada
2843,522185,VALPARAISO DE GOIAS,0,0,709,0,5,0,0,0,...,0,0,0,0,0,169,1616,Dez,2023,Quantidade aprovada
2844,522200,VIANOPOLIS,0,0,0,0,0,0,0,0,...,0,0,0,0,0,84,149,Dez,2023,Quantidade aprovada
2845,522205,VICENTINOPOLIS,0,0,11,0,0,0,0,0,...,0,0,0,0,14,0,66,Dez,2023,Quantidade aprovada
